# 爬取网易云音乐的评论

## 分析

- 打开网易云音乐，顺便选取一首歌，这里以《玫瑰花的葬礼》为例
- 打开chrome的控制台，定位到network标签的xhr选项。刷新页面。
寻找浏览器发起的请求，找到了获取评论的url为
    * `https://music.163.com/weapi/comment/resource/comments/get?csrf_token=`

    * 这是一个post请求，请求有两个参数

    ```
    params: Y6ocHTA7yqD1FLgfNri3Abpf8DWsuIucCGgGe7+JxATu4ZlZ697isicyzScAriGMAn26Dpnr5V+835DtIQB84cCjlBVXi1ikWZpd4IXdFH28rCKxvNMB00EdR95qZzxsl0to+qrkQ1vCZQ1dovJfYAenoBxaQLdpiyiQqr0SMeXn+kzIYi6Blz/PjaOK1J/wu9VnUJrV48gbkGhW2ubhi6thWGuN4pVXRx1ARLMKksctxzFmIp26tUKllnn0ixi1cHQqElyx4PlJGfyifPlsfw==
    encSecKey: 308ada056b3ed8d22babf2553bf1a1ec7f863a299604dcb9501a1235145343eeaa88aa8a707eb518334af33fb1bfef15c55451c8be3da211eec2145f3e646ae1e1ef589d2cee494ca603134f2392e9270b1c8a43c9baf920248f2dd6cb3696e654a101627100203b9252688d227782d8c9569a1b94b46fe68465a93bbfeb74eb
    ```

    发现参数不认识，猜想可能是经过加密或者时在前端页面中携带的参数。

    - 在前端页面搜索，没有发现这个两个值，排除在前端页面携带的数据，
    则这两个条请求参数是加密生成。

- 分析js加密过程。
    - 进入到该页面加载的js文件，进行局部搜索`encSecKey`
    在此处，查看的啦`encSecKey`
    ```js
    function(X7Q, e6c) {
            var i6c = {}
              , e6c = NEJ.X({}, e6c)
              , mi3x = X7Q.indexOf("?");
            if (window.GEnc && /(^|\.com)\/api/.test(X7Q) && !(e6c.headers && e6c.headers[ew9n.Bv9m] == ew9n.HS2x) && !e6c.noEnc) {
                if (mi3x != -1) {
                    i6c = j6d.gR0x(X7Q.substring(mi3x + 1));
                    X7Q = X7Q.substring(0, mi3x)
                }
                if (e6c.query) {
                    i6c = NEJ.X(i6c, j6d.fT0x(e6c.query) ? j6d.gR0x(e6c.query) : e6c.query)
                }
                if (e6c.data) {
                    i6c = NEJ.X(i6c, j6d.fT0x(e6c.data) ? j6d.gR0x(e6c.data) : e6c.data)
                }
                i6c["csrf_token"] = v7o.gN0x("__csrf");
                X7Q = X7Q.replace("api", "weapi");
                e6c.method = "post";
                delete e6c.query;
                var bWv7o = window.asrsea(JSON.stringify(i6c), bsK7D(["流泪", "强"]), bsK7D(XR0x.md), bsK7D(["爱心", "女孩", "惊恐", "大笑"]));
                e6c.data = j6d.cr8j({
                    params: bWv7o.encText,
                    encSecKey: bWv7o.encSecKey
                })
            }
    ```
    - 继续在chrome中进行断点调试: 发现此处为最终加密的结果，加密方法为`window.asrsea`
- 分析`window.asrsea`
    - 继续搜索`window.asrsea`
    - 发现了`window.asrsea = d`,所以，进行加密的函数就是`d`, 查看函数`d`
    - `encText`是经过两次`b()`函数，`encSecKey`是经过`c()`函数，
    执行的一个参数。注意其中`i`参数来源是`a(16)`
    ```js
    function a(a) {
            var d, e, b = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", c = "";
            for (d = 0; a > d; d += 1)
                e = Math.random() * b.length,
                e = Math.floor(e),
                c += b.charAt(e);
            return c
        }
        function b(a, b) {
            var c = CryptoJS.enc.Utf8.parse(b)
              , d = CryptoJS.enc.Utf8.parse("0102030405060708")
              , e = CryptoJS.enc.Utf8.parse(a)
              , f = CryptoJS.AES.encrypt(e, c, {
                iv: d,
                mode: CryptoJS.mode.CBC
            });
            return f.toString()
        }
        function c(a, b, c) {
            var d, e;
            return setMaxDigits(131),
            d = new RSAKeyPair(b,"",c),
            e = encryptedString(d, a)
        }
        function d(d, e, f, g) {
            var h = {}
              , i = a(16);
            return h.encText = b(d, g),
            h.encText = b(h.encText, i),
            h.encSecKey = c(i, e, f),
            h
        }
        function e(a, b, d, e) {
            var f = {};
            return f.encText = c(a + e, b, d),
            f
        }
    ```
    - 函数`d`中，调用了函数`a，b，c`，然后返回一个对象`h`
    - 继续查看函数`a`, 发现函数`a`随机生成了十六个字符
    - 查看函数`b`，发现函数`b`是，AES的cbc模式加密。
    - 函数`c`就是三个参数进行`RSA`加密

- 查看`window.asrsea`参数分析
    - `var bWv7o = window.asrsea(JSON.stringify(i6c), bsK7D(["流泪", "强"]), bsK7D(XR0x.md), bsK7D(["爱心", "女孩", "惊恐", "大笑"]));`
    - 继续在js代码中搜索`bsK7D`, 结果如下
    ```js
    var bsK7D = function(czA6u) {
            var m6g = [];
            j6d.bf7Y(czA6u, function(czz6t) {
                m6g.push(XR0x.emj[czz6t])
            });
            return m6g.join("")
        };
    ```
  - 可以确定，后面三个参数固定不变。
  - 继续分析及一个参数的由来

- `window.asrsea`第一个参数的由来

    - 对`window.asrsea`函数调用打断点，点击评论下一页，就可以发现
    
    ![](./网易云音乐评论爬取第一个参数.png)
    - 加密时需要使用的固定不变的参数
        - aes_key1: "0CoJUm6Qyw8W8jud"
        - aes_key2: a(16)随机生成的
        - rsa_e: 010001
        - rsa_n: 00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7
        - rsa_i: a(16)随机生成的

- 参数的传递格式，在postman中进行尝试，参数传递格式为`urlencoded`格式

In [27]:
import requests
import urllib.parse
import base64
from Cryptodome.Cipher import AES

header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36',
    # 'Postman-Token':'4cbfd1e6-63bf-4136-a041-e2678695b419',
    "origin": 'https://music.163.com',
    # 'referer':'https://music.163.com/song?id=1372035522',
    # 'accept-encoding':'gzip,deflate,br',
    'Accept': '*/*',
    'Host': 'music.163.com',
    'content-lenth': '472',
    'Cache-Control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    'Connection': 'keep-alive',
    # 'Cookie':'iuqxldmzr_=32; _ntes_nnid=a6f29f40998c88c693bc910331bd6bea,1558011234325; _ntes_nuid=a6f29f40998c88c693bc910331bd6bea; _ga=GA1.2.2120707788.1559308501; WM_TID=pV2C%2BjTrRwBBAAERUVJojniTwk8%2B8Zta; JSESSIONID-WYYY=nvf%2BggodQRfcT%2BTvBRmANqMrsDeQCxRvqwFsxDr3eJvNNWhGYFhfCXKFkfAfOdbHhpCsMzT39mAeJ7ZamBQZbiwwtnSZD%5CPWRqKxD9t6dGKD3bTVjomjgB39DB07RNIWI32bYKa2H4fg1qQgqI%2FR%2B%2Br%2BZXJvgFg1Vh%2FA2XRj9S4p0EMu%3A1560927288799; WM_NI=DthwcEQf5Ew2NbTIZmSNhSnm%2F8VWsg5RxhkYogvs2luEwZ6m5UhdzbHYPIr654ZBWKV4o22%2BEwb9BvdLS%2BFOmOAEUG%2B8xd8az4CX%2FiAL%2BZkz3syA0onCPkhQwCtL4pkUcjg%3D; WM_NIKE=9ca17ae2e6ffcda170e2e6eed2d650989c9cd1dc4bb6b88eb2c84e979f9aaff773afb6fb83d950bcb19ecce92af0fea7c3b92a88aca898e24f93bafba6f63a8ebe9caad9679192a8b4ed67ede89ab8f26df78eb889ea53adb9ba94b168b79bb9bbb567f78ba885f96a8c87a0aaf13ef7ec96a3d64196eca1d3b12187a9aedac17ea8949dccc545af918fa6d84de9e8b885bb6bbaec8db9ae638394e5bbea72f1adb7a2b365ae9da08ceb5bb59dbcadb77ca98bad8be637e2a3'
    }


def pkcs7padding(text):
    """
    明文使用PKCS7填充
    最终调用AES加密方法时，传入的是一个byte数组，要求是16的整数倍，因此需要对明文进行处理
    :param text: 待加密内容(明文)
    :return:
    """
    bs = AES.block_size  # 16
    length = len(text)
    bytes_length = len(bytes(text, encoding='utf-8'))
    # tips：utf-8编码时，英文占1个byte，而中文占3个byte
    padding_size = length if (bytes_length == length) else bytes_length
    padding = bs - padding_size % bs
    # tips：chr(padding)看与其它语言的约定，有的会使用'\0'
    padding_text = chr(padding) * padding
    return text + padding_text


def encrypt(key, content):
    """
    AES加密
    key,iv使用同一个
    模式cbc
    填充pkcs7
    :param key: 密钥
    :param content: 加密内容
    :return:
    """
    key_bytes = bytes(key, encoding='utf-8')
    iv = bytes('0102030405060708', encoding='utf-8')
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv)
    # 处理明文
    content_padding = pkcs7padding(content)
    # 加密
    encrypt_bytes = cipher.encrypt(bytes(content_padding, encoding='utf-8'))
    # 重新编码
    result = str(base64.b64encode(encrypt_bytes), encoding='utf-8')
    return result


def getcomment(songid, page):
    url = "https://music.163.com/weapi/v1/resource/comments/R_SO_4_" + songid + "?csrf_token="
    print(url)
    formdata = {
        "params": "",
        "encSecKey": "c81160c64a08feb6cfed91c1619d5bffd05dd278b685c94a748689edf035ee0436b66aa7019927ce0fedd26aee9a22cdc6743e58a120f9db0126ebb2e61dae3f7ee21088eb747f829bceed9a5bbb9ee7a2eecf1a358feac431acaab17c95b8491a6a955f7c17a02a3e7886390c2cb3b981f4ccbd5163a566d27ace95db073401",
    }

    aes_key = '0CoJUm6Qyw8W8jud'  ## 不变的
    print('第一次加密的key:' + aes_key)
    # 对英文加密
    source_en = '{"rid":"R_SO_4_' + songid + '","offset":"' + str(
        page * 20) + '","total":"false","limit":"20","csrf_token":""}'

    # offset自己该
    print(source_en)
    encrypt_en = encrypt(aes_key, source_en)  # 第一次加密
    print("第一次加密后的params: ", encrypt_en)
    aes_key = '3Unu7SzdXGctW1vA'
    print("第二次加密的key: ", aes_key)
    encrypt_en = encrypt(aes_key, str(encrypt_en))  # 第二次加密
    print("第二次加密后的params: ", encrypt_en)
    formdata['params'] = encrypt_en
    print("参数params", formdata['params'])
    formdata = urllib.parse.urlencode(formdata).encode('utf-8')
    print("url编码后的params", formdata)
    req = requests.post(url=url, data=formdata, headers=header)
    return req.json()


if __name__ == '__main__':
    songid = '27646687'
    page = 0
    text = ''
    for page in range(1, 3):
        comment = getcomment(songid, page)
        print(comment)

https://music.163.com/weapi/v1/resource/comments/R_SO_4_27646687?csrf_token=
第一次加密的key:0CoJUm6Qyw8W8jud
{"rid":"R_SO_4_27646687","offset":"20","total":"false","limit":"20","csrf_token":""}
第一次加密后的params:  i7wvj8/cCucRRvz8Gr5sRjgUm7n/3oXYf5BoYKIXYVSjr5jxs4UO9mT1jGWBOEiW2usHgf+gNyJcbEZGEStT8nnvPhB0nbuSeAdze0mQ4MRJDiEjhBk4xudUO1uUIqVr
第二次加密的key:  3Unu7SzdXGctW1vA
第二次加密后的params:  VsNo/1OElaSnU2wVOdLpj1y48/U550HvS6s46vFiLhzXifErk3cQBHBCDjhVYvS729DjqzxO4A69VTLJKw6Fzat2C9MvkK50ggyI4yDpnsewKKajriIA6WfUv9aS9C4CBTp+XdqsNIpq+3VZ5MrG99FsEGsJIb9jxFtX1pKeffAb9sFrDsg+SYrHOyv7XVbF
参数params VsNo/1OElaSnU2wVOdLpj1y48/U550HvS6s46vFiLhzXifErk3cQBHBCDjhVYvS729DjqzxO4A69VTLJKw6Fzat2C9MvkK50ggyI4yDpnsewKKajriIA6WfUv9aS9C4CBTp+XdqsNIpq+3VZ5MrG99FsEGsJIb9jxFtX1pKeffAb9sFrDsg+SYrHOyv7XVbF
url编码后的params b'params=VsNo%2F1OElaSnU2wVOdLpj1y48%2FU550HvS6s46vFiLhzXifErk3cQBHBCDjhVYvS729DjqzxO4A69VTLJKw6Fzat2C9MvkK50ggyI4yDpnsewKKajriIA6WfUv9aS9C4CBTp%2BXdqsNIpq%2B3VZ5MrG99FsEGsJIb9jxFtX1pKeffAb9sFrDsg%2BSYrHOyv7XVbF&

SSLError: HTTPSConnectionPool(host='music.163.com', port=443): Max retries exceeded with url: /weapi/v1/resource/comments/R_SO_4_27646687?csrf_token= (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:852)'),))